In [ ]:
import pandas as pd
import numpy as np
from time import time
import sys, io
import larch as lx
from pytest import approx

In [ ]:
varnames = [
    'price', 'time', 'conven', 'comfort', 'meals', 'petfr',
    'emipp', 'nonsig1', 'nonsig2', 'nonsig3',
]
d = lx.examples.ARTIFICIAL()
m = lx.Model(d)
m.utility_ca = sum(lx.PX(i) for i in varnames)
m.choice_ca_var = 'choice'
randvars_normal = ['meals', 'petfr', 'emipp']

In [ ]:
m.mixtures = [lx.mixtures.Normal(k, f"sd.{k}") for k in randvars_normal]

In [ ]:
m.n_draws = 300
m.seed = 42
m.prerolled = True
m.common_draws = True

In [ ]:
# TEST
assert m.loglike() == approx(-4394.45556640625)

In [ ]:
# TEST
assert m._draws[0:2] == approx(np.array([[ 0.328483,  0.256732,  0.388444],
       [ 0.170979,  0.871594,  0.298636]], dtype=np.float32), rel=1e-5)
assert m._draws[-2:] == approx(np.array([[ 0.300983,  0.328402,  0.055866],
       [ 0.363251,  0.119268,  0.45048 ]], dtype=np.float32), rel=1e-5)

In [ ]:
result = m.maximize_loglike(stderr=True)

In [ ]:
# TEST
assert m.n_draws == 300
assert m._draws.shape == (300, 3) # common draws
assert result.loglike == approx(-2278.119873)
assert result.x == approx(np.array([ 
    1.084783,  0.878294, -2.071825,  1.709024,  0.064876,  0.0095  ,  0.01686 ,  3.960227, -1.050591,  1.042768,
       -0.703356, -1.549873, -1.467843]), rel=1e-4)
assert result.stderr == approx(np.array([ 
    0.198335,  0.155281,  0.232656,  0.209664,  0.147426,  0.152424,  0.134502,  0.399234,  0.179498,  0.14787 ,
        0.220511,  0.3684  ,  0.177694], ), rel=1e-4)

In [ ]:
# TEST
# from io import StringIO
# xlogit_ref_results = pd.read_csv(StringIO('''
# Coefficient,   Estimate ,  Std.Err. ,      z-val ,    P>|z|
# price      , -1.0320343 , 0.1553819 , -6.6419223 , 2.36e-10
# time       , -1.4595725 , 0.1690836 , -8.6322536 , 7.34e-17
# conven     ,  0.8895740 , 0.1413429 ,  6.2937284 , 2.19e-09
# comfort    ,  1.0633392 , 0.1825744 ,  5.8241414 , 3.68e-08
# meals      ,  1.7115916 , 0.1966436 ,  8.7040298 , 3.99e-17
# petfr      ,  3.8752937 , 0.3735491 , 10.3742563 , 6.83e-24
# emipp      , -2.0163539 , 0.2139036 , -9.4264592 ,  6.5e-20
# nonsig1    ,  0.0725413 , 0.1268775 ,  0.5717423 ,    0.678
# nonsig2    ,  0.0160257 , 0.1339144 ,  0.1196709 ,    0.792
# nonsig3    ,  0.0189635 , 0.1130936 ,  0.1676797 ,    0.787
# sd.meals   , -0.6844918 , 0.2367959 , -2.8906402 ,   0.0123
# sd.petfr   , -1.3225227 , 0.3194845 , -4.1395524 , 0.000154
# sd.emipp   ,  1.0039927 , 0.1420153 ,  7.0696075 ,  1.3e-11
# '''), skipinitialspace=True, index_col=0)
# xlogit_ref_results.index = xlogit_ref_results.index.str.strip()

# pd.testing.assert_series_equal(
#     xlogit_ref_results['Estimate '].sort_index(),
#     pd.Series(dict(zip(m.pnames, m.pvals))).sort_index(),
#     check_names=False,
#     rtol=0.4,
# )

In [ ]:
m2 = m.copy()
m2.pvals = 'init'
m2.common_draws = False

In [ ]:
result2 = m2.maximize_loglike(stderr=True)

In [ ]:
# TEST
# changing seeds
m.seed = 45
assert m.loglike() == approx(-2279.95263671875)
m.seed = 46
assert m.loglike() == approx(-2282.3642578125)